In [ ]:
'''
Posibles hipotesis: 

country, activity, species involve, 

Most of the incidents in the Global Shark Attack File have nothing to do with predation on humans

A very small percentage of shark species, about two dozen, are considered potentially dangerous to humans
because of their size and dentition. Yet each year, for every human killed by a shark, 
our species slaughters millions of sharks - about 73 million sharks last year

Most of the incidents in the Global Shark Attack File have nothing to do with predation on humans. 
Some accidents are motivated by a displacement or territorial behavior when a shark feels threatened; 
still others are the result of the shark responding to sensory predatory input 
(i.e., overwhelmed by the presence of many fishes) and environmental conditions (murky water), 
which may cause them to respond in a reflexive response to stimuli. 
Sharks also exhibit curiosity and may investigate unknown or unfamiliar objects; 
they learn by exploring their environment, and - lacking hands - they use their mouths and teeth to examine unfamiliar objects. 

When a shark grabs a person by the hand/wrist and tows them along the surface, 
tosses a surfboard (or a Frisbee as in case 1968.08.24) it is probably "play behavior", not aggression. 
How can case GSAF 1971.04.11 which the swimmer was repeatedly bitten by a large shark and 
case 1985.01.04 in which the diver's injury necessitated a Band-aid be compared? It is akin to comparing a 
head-on high-speed vehicular collision with a shopping cart ding on the door of a parked car. Global Shark Attack File believes 
the only way to sort fact from hype is by forensic examination of each incident. 

--- Una de las hipotesis, las especies más dañinas
--- Hay más "accidentes" que ataques puros y duros

tips:

- Sacar con regular expressions el tipo de tiburones
- Me tengo que centrar en la columna type y en la columna Species también en injury y fatal
- 

'''

In [16]:
import pandas as pd

In [17]:
data = pd.read_csv('GSAF5.csv', encoding = "ISO-8859-1")

In [ ]:
# Operaciones genéricas para conocer los datos

In [18]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [19]:
data.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN
5991,ND.0001,1845-1853,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2,NaN,NaN


In [20]:
data.shape

(5992, 24)

In [21]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [ ]:
# qué columnas quiero para mi investigación
''' Case Number, year, tipo, country, area, location, activity, sex, injury, fatal, species

'''

In [ ]:
# Date o year

In [22]:
data["Date"]

0         18-Sep-16
1         18-Sep-16
2         18-Sep-16
3         17-Sep-16
4         16-Sep-16
           ...     
5987    Before 1903
5988    Before 1903
5989      1900-1905
5990      1883-1889
5991      1845-1853
Name: Date, Length: 5992, dtype: object

In [23]:
data["Year"]

0       2016
1       2016
2       2016
3       2016
4       2016
        ... 
5987       0
5988       0
5989       0
5990       0
5991       0
Name: Year, Length: 5992, dtype: int64

In [23]:
# Para ver los valores únicos de la columna
#set(data["Year"])

# tengo que obtener el índice del primer 1989, y de ahí empezar a borrar filas


In [24]:
data.columns

remove_columns = ["Date", "Name", "Age", "Time", "Investigator or Source", "pdf",
                  "href formula", "href", "Case Number.1", "Case Number.2", 
                  "original order", "Unnamed: 22", "Unnamed: 23"]

clean_data = data.drop(columns=remove_columns)
clean_data.head()


,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Species
0,2016.09.18.c,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Minor injury to thigh,N,NaN
1,2016.09.18.b,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to hands,N,NaN
2,2016.09.18.a,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to lower leg,N,NaN
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,Struck by fin on chest & leg,N,NaN
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,No injury: Knocked off board by shark,N,2 m shark


In [26]:
clean_data.shape

(5992, 11)

In [33]:
# quitar las filas que no quiero...
clean_data = clean_data.loc[clean_data["Year"] > 1989]
clean_data.shape


(2423, 11)

In [34]:
clean_data.tail()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Species
2419,1990.03.05,1990,Unprovoked,REUNION,Sainte-Marie,Baie de la Mare,Surfing,M,Survived,N,3 m [10'] bull shark
2420,1990.02.17,1990,Unprovoked,USA,Hawaii,"Mokapu, Kane'ohe Marine Air Corps Station, O'ahu",Scuba diving & spearfishing,M,Right arm severed FATAL,Y,Two sharks seen in vicinity: 2.4 m & 4.25 m [...
2421,1990.02.05,1990,Unprovoked,USA,Florida,"Monster Hole, Sebastian Inlet, Indian River Co...",Board sailing,M,Foot bitten,N,NaN
2422,1990.01.12,1990,Unprovoked,USA,California,"Montera Beach, San Mateo County",Surfing (sitting on his board),M,"No injury, board bitten",N,3 m to 4 m [10' to 13'] white shark
2423,1990.00.00,1990,Unprovoked,USA,Florida,"Pensacola, Escambia County",Surfing,M,NaN,UNKNOWN,NaN


In [ ]:
# Primera hipotésis
# Ver qué tipo de valores contiene la columna type para analizar ante que tipo de sucesos
# qué se puede considerar accidente, ataque en sí, o qué otro tipo de sucesos
# somos más irresponsables
# nos bañamos más y por eso se han multiplicado los sucesos
# son los hombres o son las mujeres más imprudentes
#

In [35]:
# Cuál elijo y por qué. En un primer momento se podría decir que tendría que optar por date,
# porque al parecer hay más información pero before 1923, seguro que hubo más,
# pero no se han contabilizado. Por tanto, el criterio temporal va a ser a partir del año
# en el que la recogida de datos es sistemática, se ve una documentación detrás,
# Use these data with caution, remembering that scientific and media coverage of shark attack 
# during the early part of this century was far less inclusive than that today. 
# The apparent drop in number of attacks in the 1970’s and 1980’s 
# is in part reflective of the largely inactive state of the ISAF during those decades.
# Me voy a centrar a partir de la década de los 90, casi 30 años...

# Tomada esa decisión, eliminar las filas más antiguas

# tengo que limpiar columna type, columna activity, sex, injury, fatal

null_cols = clean_data.isnull().sum()
null_cols[null_cols > 0]


Country          1
Area            97
Location       102
Activity       119
Sex            147
Injury           5
Fatal (Y/N)      2
Species        869
dtype: int64

In [37]:
clean_data.columns

Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex ', 'Injury', 'Fatal (Y/N)', 'Species '],
      dtype='object')

In [41]:
set(clean_data["Type"])

{'Boat', 'Invalid', 'Provoked', 'Sea Disaster', 'Unprovoked'}

In [ ]:
'''

“Unprovoked attacks” are defined as incidents where an attack on a live human occurs 
in the shark’s natural habitat with no human provocation of the shark.

“Provoked attacks” occur when a human initiates interaction with a shark in some way. 
These include instances when divers are bitten after harassing or trying to touch sharks, 
attacks on spearfishers, attacks on people attempting to feed sharks, 
bites occurring while unhooking or removing a shark from a fishing net, etc.

'''

In [40]:
type_attacks = clean_data.groupby("Type")
type_attacks.describe()

Year                                                          \
               count         mean       std     min      25%     50%     75%   
Type                                                                           
Boat            98.0  2006.020408  7.513534  1990.0  2000.00  2007.0  2014.0   
Invalid        180.0  2005.972222  6.720627  1990.0  2001.75  2007.0  2011.0   
Provoked       172.0  2006.819767  6.495798  1990.0  2003.00  2008.0  2012.0   
Sea Disaster    19.0  2004.421053  6.568577  1993.0  2000.50  2006.0  2008.0   
Unprovoked    1954.0  2005.096213  7.348675  1990.0  1999.00  2006.0  2011.0   

                      
                 max  
Type                  
Boat          2016.0  
Invalid       2016.0  
Provoked      2016.0  
Sea Disaster  2014.0  
Unprovoked    2016.0

In [ ]:
# Limpieza de la columna "Sex"

In [44]:
clean_data["Sex"] = clean_data["Sex "]

set(clean_data["Sex"])

{'F', 'M', 'M ', 'lli', nan}

In [50]:
clean_data ["Sex"] = clean_data["Sex"].replace("M ", "M")

In [51]:
set(clean_data["Sex"])

{'F', 'M', 'lli', nan}

In [53]:
clean_data.loc[clean_data["Sex"] == "lli"]


,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Species,Sex,Fatal
1400,2004.11.11.b,2004,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,lli,"Lacerations to hand, knee & thigh",N,5.5 m [18'] white shark,lli,N


In [55]:
clean_data["Sex"] = clean_data["Sex"].fillna(0)
set(clean_data["Sex"])

{0, 'F', 'M', 'lli'}

In [57]:
clean_data["Sex"] = clean_data["Sex"].replace("lli", 0)
set(clean_data["Sex"])
# Concluida la limpieza de la fila "Sex"

{0, 'F', 'M'}

In [ ]:
# Limpieza de la columna "Fatal"

In [46]:
clean_data["Fatal"] = clean_data["Fatal (Y/N)"]

set(clean_data["Fatal"])

{'N', 'UNKNOWN', 'Y', nan}

In [59]:
clean_data["Fatal"] = clean_data["Fatal"].replace('UNKNOWN', 0)
clean_data["Fatal"] = clean_data["Fatal"].fillna(0)
set(clean_data["Fatal"])

{0, 'N', 'Y'}

In [47]:
set(clean_data["Injury"])

#bitten

#no injury

{nan,
 'Ankle & foot bitten',
 'No injury, but sharks repeatedly hit their fins and guns',
 'Leg bitten by shark that had been shot in the head by another diver PROVOKED INCIDENT',
 'Thigh bitten',
 'Avulsion injury  to lower right leg',
 'Hand & foot bitten',
 'Some of the bodies recovered had been bitten by sharks',
 'Glancing bite to wrist from netted shark PROVOKED INCIDENT',
 'No injury, shark bit surfboard ',
 'Leg & board bitten',
 '2 puncture wounds in left leg',
 'FATAL, head, shoulder and arm severed, remains recovered at Munglinup Beach on 9/13/1995',
 'No injury to occupant, shark removed small auxiliary outboard motor',
 'Puncture wounds on ankle & leg',
 'No injury from shark',
 'Laceration to foot ',
 'Wrist bitten by hooked shark PROVOKED INCIDENT',
 'Injuries to face & neck PROVOKED INCIDENT',
 'Presumed to have drowned until his body washed ashore 2 weeks later with shark bites',
 'No injury, hooked shark bit their 4.8 m inflatable boat',
 'Laceration to lower leg',
 

In [ ]:
# operaciones para la limpieza de datos y para la manipulación.
# El ejercicio tiene que tener al menos ocho operaciones


'''
- comprobar los valores nulos con isnull
- con drop borramos directamente las columnas que queramos
- rellenar celdas con ceros con fillna(0)
- modficar strings
- encontrar duplicados y borrarlos
- renombrar columnas
- cambiar el orden de las columnas
- filtrar los datos
- crear categorías adiccionales
- fusionar tablas

'''

In [36]:
data["Type"]

0       Unprovoked
1       Unprovoked
2       Unprovoked
3       Unprovoked
4       Unprovoked
           ...    
5987    Unprovoked
5988    Unprovoked
5989    Unprovoked
5990    Unprovoked
5991    Unprovoked
Name: Type, Length: 5992, dtype: object

0       Unprovoked
1       Unprovoked
2       Unprovoked
3       Unprovoked
4       Unprovoked
           ...    
5987    Unprovoked
5988    Unprovoked
5989    Unprovoked
5990    Unprovoked
5991    Unprovoked
Name: Type, Length: 5992, dtype: object

Year                                                   \
               count         mean         std     min     25%     50%   
Type                                                                    
Boat           200.0  1963.745000  200.649198     0.0  1962.0  1989.0   
Boating        110.0  1927.945455   24.771186  1865.0  1923.0  1936.0   
Invalid        519.0  1952.616570  131.377839     0.0  1931.5  1967.0   
Provoked       557.0  1943.211849  222.047458     0.0  1949.0  1965.0   
Sea Disaster   220.0  1890.800000  319.539403     0.0  1924.5  1944.0   
Unprovoked    4386.0  1919.573643  310.809603     0.0  1944.0  1982.0   

                              original order                                   \
                  75%     max          count         mean          std    min   
Type                                                                            
Boat          2006.25  2016.0          200.0  3650.385000  1393.716793   29.0   
Boating       1946.00  1956.0          110.0  1250.554545   478.362605  314.0   
Invalid       2003.00  2016.0          519.0  2837.283237  1733.515091   71.0   
Provoked      2001.00  2016.0          557.0  2894.405745  1568.017245   33.0   
Sea Disaster  1966.00  2014.0          220.0  1910.254545  1240.867821   31.0   
Unprovoked    2004.00  2016.0         4386.0  3098.195394  1755.297468    2.0   

                                                
                  25%     50%      75%     max  
Type                                            
Boat          2435.75  3530.0  4739.25  5988.0  
Boating       1015.00  1342.0  1618.75  1968.0  
Invalid       1228.00  2726.0  4432.00  5943.0  
Provoked      1710.00  2576.0  4314.00  5970.0  
Sea Disaster  1054.50  1571.0  2640.50  5749.0  
Unprovoked    1577.25  3240.5  4589.50  5993.0

In [16]:
#Eliminar el espacio en el valor Sea Disaster

In [11]:
boat = []
boating = []
invalid = []
provoked = []
sea_disaster = []
unprovoked = []

for e in data["Type"]:
    if e == "Boat":
        boat.append(e)
    elif e == "Boating":
        boating.append(e)
    elif e == "Invalid":
        invalid.append(e)
    elif e == "Provoked":
        provoked.append(e)
    elif e == "Sea Disaster":
        sea_disaster.append(e)
    elif e == "Unprovoked":
        unprovoked.append(e)

In [15]:
print(len(boat))
print(len(boating))
print(len(invalid))
print(len(provoked))
print(len(sea_disaster))
print(len(unprovoked))

200
110
519
557
220
4386


In [ ]:
# Qué significa cada tipo. Por lo que he podido leer en la web 

In [ ]:
# Segunda hipotésis

In [24]:
# Eliminar el espacio en blanco de la columna Species
data["Species"] = data["Species "]

In [28]:
data["Species"]

0             NaN
1             NaN
2             NaN
3             NaN
4       2 m shark
          ...    
5987          NaN
5988          NaN
5989          NaN
5990          NaN
5991          NaN
Name: Species, Length: 5992, dtype: object

In [29]:
sharks_species = data.groupby("Species")